In [3]:
import pandas as pd
import plotly.express as px
df = pd.read_csv(
    '/home/parker/Documents/Tokemak/tokemak-analytics/.data/lsd_eth/daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv', index_col=0, parse_dates=True)
df.columns

Index(['lp_token_total_supply', 'lp_token_virtual_price', 'pool_balance_0',
       'pool_balance_1', 'block', 'pool_balance_market_0',
       'pool_balance_market_1', 'market_eth_tvl',
       'pool_balance_market_portion_0', 'pool_balance_market_portion_1',
       'lp_token_market_eth_value', 'pool_balance_equilibrium_0',
       'pool_balance_equilibrium_1', 'equilibrium_eth_tvl',
       'pool_balance_equilibrium_portion_0',
       'pool_balance_equilibrium_portion_1', 'lp_token_equilibrium_eth_value',
       'lp_token_discount_percent', 'raw_base_apr', 'raw_fee_apr',
       'raw_incentive_apr', 'raw_price_apr', 'tokens', 'pool_address'],
      dtype='object')

In [6]:
import plotly.express as px
px.bar(df[['pool_balance_equilibrium_portion_0', 'pool_balance_equilibrium_portion_1']].dropna(), title='stETH ETH reserve ratio')

In [27]:
from glob import glob
import pandas as pd

import plotly.express as px
paths = glob('/home/parker/Documents/Tokemak/tokemak-analytics/.data/lsd_eth/daily_snapshots/*.csv')
df = pd.concat([pd.read_csv(p, parse_dates=True, index_col=0)['raw_incentive_apr'] for p in paths], axis=1)
df.columns = [pd.read_csv(p, parse_dates=True, index_col=0)['pool_address'][0] for p in paths]
px.line(df)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df.dtypes

In [7]:
dfs[0].columns

Index(['lp_token_total_supply', 'lp_token_virtual_price', 'pool_balance_0',
       'pool_balance_1', 'block', 'pool_balance_market_0',
       'pool_balance_market_1', 'market_eth_tvl',
       'pool_balance_market_portion_0', 'pool_balance_market_portion_1',
       'lp_token_market_eth_value', 'pool_balance_equilibrium_0',
       'pool_balance_equilibrium_1', 'equilibrium_eth_tvl',
       'pool_balance_equilibrium_portion_0',
       'pool_balance_equilibrium_portion_1', 'lp_token_equilibrium_eth_value',
       'lp_token_discount_percent', 'raw_base_apr', 'raw_fee_apr',
       'raw_incentive_apr', 'raw_price_apr', 'tokens', 'pool_address'],
      dtype='object')

In [16]:
import pandas as pd


SECONDS_IN_A_YEAR = 31_536_000


def get_rows_per_year(df: pd.DataFrame) -> float:
    """Returns the number of rows in a year"""
    cur_timestamp = df.index.values[-1]
    prev_timestamp = df.index.values[-2]
    diff = (cur_timestamp - prev_timestamp).astype("timedelta64[s]").astype(int)
    if diff <= 0:
        raise ValueError("diff cannot be negative")
    return SECONDS_IN_A_YEAR / diff


def compute_annualized_percent_change(df: pd.DataFrame, col_name: str) -> pd.Series:
    if col_name not in df.columns:
        raise ValueError(f"Could not find {col_name=} in {df.columns=}")

    rows_per_year = get_rows_per_year(df)
    diff = df[col_name] - df[col_name].shift(1)
    return 100 * rows_per_year * diff / df[col_name].shift(1)


In [17]:
df['annulaized_change_in_vp'] = compute_annualized_percent_change(df, 'lp_token_virtual_price')

In [18]:
df['fee_and_base_apr'] = df[['fee_apr', 'base_apr']].sum(axis=1)
df['fee_and_base_apr']

timestamp
2022-12-02    0.000000
2022-12-03    1.301661
2022-12-04    0.576469
2022-12-05    0.510239
2022-12-06    0.799277
                ...   
2023-04-27    1.069691
2023-04-28    1.258642
2023-04-29    1.088108
2023-04-30    1.874683
2023-05-01    1.104849
Name: fee_and_base_apr, Length: 151, dtype: float64

In [29]:
df['diff'] = df['fee_and_base_apr'] - df['annulaized_change_in_vp']
px.line(df[['diff', 'fee_apr']] )

In [28]:
px.line(df[['diff', 'fee_and_base_apr']] )

In [ ]:
px.line(df[['fee_and_base_apr', 'annulaized_change_in_vp']])

In [20]:
px.line(df[['fee_and_base_apr', 'annulaized_change_in_vp']])

In [2]:
px.line(df['lp_token_virtual_price'].diff())